In [1]:
import numpy as np
from scipy.interpolate import interp1d as interp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from src.thunderflask import thunderflask
from src.bacteria import strain
from src.ffgen import ffgen
from src.ffgen_isaac import ffgen
from src.redGen import reductionist
from src.codonTable import codonTable
from src.codonUtils import utils
from src.bacteria import strain
import pickle
from src.codonOptimizer import tableOptimizer
from queue import Queue as queue
from random import choice
from matplotlib.colors import LinearSegmentedColormap as LSC
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cProfile
import copy
import random

In [2]:
#Import reductionist table
with open('res/ff_table_manuscript.pickle', 'rb') as handle:
    ff_table = pickle.load(handle)

In [3]:
ffTable = codonTable(ff_table)

In [4]:
ffTable.dictToTable()

U        C        A        G
U U  UUU : *  UCU : *  UAU : Y  UGU : *
  C  UUC : C  UCC : *  UAC : *  UGC : *
  A  UUA : *  UCA : W  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : E  UGG : V
C U  CUU : A  CCU : P  CAU : *  CGU : *
  C  CUC : *  CCC : D  CAC : *  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : K
  G  CUG : *  CCG : *  CAG : S  CGG : *
A U  AUU : Q  ACU : *  AAU : *  AGU : I
  C  AUC : *  ACC : *  AAC : T  AGC : *
  A  AUA : G  ACA : *  AAA : *  AGA : *
  G  AUG : *  ACG : R  AAG : *  AGG : *
G U  GUU : *  GCU : H  GAU : *  GGU : *
  C  GUC : *  GCC : *  GAC : *  GGC : L
  A  GUA : *  GCA : M  GAA : F  GGA : *
  G  GUG : N  GCG : *  GAG : *  GGG : *

In [5]:
with open('res/ff16table_manuscript.pickle', 'rb') as handle:
    ff16_table = pickle.load(handle)

In [6]:
ff16Table = codonTable(ff16_table)

In [7]:
ff16Table.dictToTable()

U        C        A        G
U U  UUU : A  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : Q  UAC : *  UGC : *
  A  UUA : *  UCA : *  UAA : K  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : R
C U  CUU : *  CCU : S  CAU : *  CGU : *
  C  CUC : *  CCC : *  CAC : Y  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : V
  G  CUG : G  CCG : *  CAG : *  CGG : *
A U  AUU : *  ACU : *  AAU : I  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : L
  A  AUA : E  ACA : *  AAA : *  AGA : *
  G  AUG : *  ACG : D  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : *  GGU : N
  C  GUC : F  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : T  GAA : *  GGA : *
  G  GUG : *  GCG : *  GAG : P  GGG : *

In [8]:
ff17_table = copy.deepcopy(ff16_table)

In [9]:
ff17_table['UUA'] = 'M'

In [10]:
ff17Table = codonTable(ff17_table)

In [11]:
ff17Table.dictToTable()

U        C        A        G
U U  UUU : A  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : Q  UAC : *  UGC : *
  A  UUA : M  UCA : *  UAA : K  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : R
C U  CUU : *  CCU : S  CAU : *  CGU : *
  C  CUC : *  CCC : *  CAC : Y  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : V
  G  CUG : G  CCG : *  CAG : *  CGG : *
A U  AUU : *  ACU : *  AAU : I  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : L
  A  AUA : E  ACA : *  AAA : *  AGA : *
  G  AUG : *  ACG : D  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : *  GGU : N
  C  GUC : F  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : T  GAA : *  GGA : *
  G  GUG : *  GCG : *  GAG : P  GGG : *

In [12]:
with open('res/ff17_table.pickle', 'wb') as handle:
    pickle.dump(ff17_table, handle)

In [13]:
with open('res/ff17_table.pickle', 'rb') as handle:
    ff17 = pickle.load(handle)

In [14]:
ff18_table = copy.deepcopy(ff17)

In [15]:
ff18_table['CCC'] = 'W'

In [16]:
ff18Table = codonTable(ff18_table)

In [17]:
ff18Table.dictToTable()

U        C        A        G
U U  UUU : A  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : Q  UAC : *  UGC : *
  A  UUA : M  UCA : *  UAA : K  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : R
C U  CUU : *  CCU : S  CAU : *  CGU : *
  C  CUC : *  CCC : W  CAC : Y  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : V
  G  CUG : G  CCG : *  CAG : *  CGG : *
A U  AUU : *  ACU : *  AAU : I  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : L
  A  AUA : E  ACA : *  AAA : *  AGA : *
  G  AUG : *  ACG : D  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : *  GGU : N
  C  GUC : F  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : T  GAA : *  GGA : *
  G  GUG : *  GCG : *  GAG : P  GGG : *

In [18]:
with open('res/ff18_table.pickle', 'wb') as handle:
    pickle.dump(ff18_table, handle)

In [19]:
with open('res/ff18_table.pickle', 'rb') as handle:
    ff18 = pickle.load(handle)

In [20]:
ff19_table = copy.deepcopy(ff18)

In [21]:
ff19_table['AGG'] = 'H'

In [22]:
ff19Table = codonTable(ff19_table)

In [23]:
ff19Table.dictToTable()

U        C        A        G
U U  UUU : A  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : Q  UAC : *  UGC : *
  A  UUA : M  UCA : *  UAA : K  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : R
C U  CUU : *  CCU : S  CAU : *  CGU : *
  C  CUC : *  CCC : W  CAC : Y  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : V
  G  CUG : G  CCG : *  CAG : *  CGG : *
A U  AUU : *  ACU : *  AAU : I  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : L
  A  AUA : E  ACA : *  AAA : *  AGA : *
  G  AUG : *  ACG : D  AAG : *  AGG : H
G U  GUU : *  GCU : *  GAU : *  GGU : N
  C  GUC : F  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : T  GAA : *  GGA : *
  G  GUG : *  GCG : *  GAG : P  GGG : *

In [24]:
with open('res/ff19_table.pickle', 'wb') as handle:
    pickle.dump(ff19_table, handle)

In [25]:
with open('res/ff19_table.pickle', 'rb') as handle:
    ff19 = pickle.load(handle)

In [26]:
ff20_table = copy.deepcopy(ff19)

In [27]:
ff20_table['GAU'] = 'C'

In [28]:
ff20Table = codonTable(ff20_table)

In [29]:
ff20Table.dictToTable()

U        C        A        G
U U  UUU : A  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : Q  UAC : *  UGC : *
  A  UUA : M  UCA : *  UAA : K  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : R
C U  CUU : *  CCU : S  CAU : *  CGU : *
  C  CUC : *  CCC : W  CAC : Y  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : V
  G  CUG : G  CCG : *  CAG : *  CGG : *
A U  AUU : *  ACU : *  AAU : I  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : L
  A  AUA : E  ACA : *  AAA : *  AGA : *
  G  AUG : *  ACG : D  AAG : *  AGG : H
G U  GUU : *  GCU : *  GAU : C  GGU : N
  C  GUC : F  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : T  GAA : *  GGA : *
  G  GUG : *  GCG : *  GAG : P  GGG : *

In [30]:
with open('res/ff20_table.pickle', 'wb') as handle:
    pickle.dump(ff20_table, handle)